In [3]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [4]:
train_data_path = "../../data/garments_train.csv"
test_data_path = "../../data/garments_test.csv"

### 리뷰에 내용이 없는데 무조건 긍정처리하는 경향이 있어 이를 수정하고자 함.
예: 가격이 비싸지만 디자인이 이뻐요. 사이즈? 긍정!

In [23]:
base_df = pd.read_csv(train_data_path).loc[:, ["RawText", "Aspect", "SentimentPolarity"]].drop_duplicates(subset=["RawText", "Aspect"]).reset_index(drop=True)
base_df.head(2)

,RawText,Aspect,SentimentPolarity
0,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,사이즈,1
1,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,사이즈,-1


In [24]:
target_df = base_df.pivot_table(values="SentimentPolarity", index="RawText", columns="Aspect").fillna(0).reset_index()

target_list = []
for idx, row in target_df.iterrows():
    for aspect in "가격", "기능", "디자인", "사이즈", "품질":
        target_list.append({
            "RawText": row["RawText"],
            "Aspect": aspect,
            "SentimentPolarity": row[aspect]
        })

train_df = pd.DataFrame(target_list)
train_df.head(10)

,RawText,Aspect,SentimentPolarity
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,가격,1.0
1,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,기능,0.0
2,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,디자인,1.0
3,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,사이즈,0.0
4,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,품질,0.0
5,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,가격,0.0
6,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,기능,0.0
7,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,디자인,0.0
8,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,사이즈,0.0
9,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,품질,-1.0


In [25]:
train_df["SentimentPolarity"].value_counts()

 0.0    100967
 1.0     30895
-1.0     10933
Name: SentimentPolarity, dtype: int64

In [26]:
label_encoder = LabelEncoder()
enc_data = label_encoder.fit_transform(train_df["SentimentPolarity"])
num_labels = len(set(enc_data))

In [27]:
label_items = label_encoder.classes_
label_numbers = label_encoder.transform(label_items)
dict(zip(label_items, label_numbers))

{-1.0: 0, 0.0: 1, 1.0: 2}

In [28]:
X_train_text = train_df.loc[:, "RawText"].to_list()
X_train_pair = train_df.loc[:, "Aspect"].to_list()
y_train = enc_data

In [29]:
HUGGING_FACE_PATH = "klue/bert-base"
model = TFAutoModelForSequenceClassification.from_pretrained(HUGGING_FACE_PATH, num_labels=num_labels, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_PATH)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
X_train_encoding = tokenizer(
    text=X_train_text,
    text_pair=X_train_pair,
    padding=True,
    truncation=True,
    max_length=42,
    return_token_type_ids=True
)

In [31]:
SHUFFLE_PARAM = 1000

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoding),
    y_train
)).shuffle(SHUFFLE_PARAM)

In [32]:
BATCH_PARAM = 32

validation_length = int(len(train_df) // 10)
train_except_val = train_dataset.skip(validation_length).batch(BATCH_PARAM)
validation_data = train_dataset.take(validation_length).batch(BATCH_PARAM)

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 110619651 (421.98 MB)
Trainable params: 110619651 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
model.fit(
    train_except_val,
    epochs=1,
    batch_size=BATCH_PARAM,
    validation_data=validation_data)

4017/4017 [==============================] - 1305s 309ms/step - loss: 0.2942 - accuracy: 0.8976 - val_loss: 0.2533 - val_accuracy: 0.9139


In [35]:
test_df = pd.read_csv(test_data_path).loc[:, ["RawText", "Aspect", "SentimentPolarity"]].drop_duplicates().reset_index(drop=True)
test_df.head(2)

,RawText,Aspect,SentimentPolarity
0,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,품질,-1
1,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,기능,-1


In [36]:
X_test_text = test_df.loc[:, "RawText"].to_list()
X_test_pair = test_df.loc[:, "Aspect"].to_list()
y_test = label_encoder.transform(test_df.loc[:, "SentimentPolarity"].to_list())

In [37]:
X_test_encoding = tokenizer(
    text=X_test_text,
    text_pair=X_test_pair,
    padding=True,
    truncation=True,
    max_length=42,
    return_token_type_ids=True
)

In [38]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    dict(X_test_encoding)
).batch(BATCH_PARAM)

In [39]:
predictions = model.predict(test_dataset)
predictions.logits

351/351 [==============================] - 41s 106ms/step


array([[ 3.2171416 , -1.0164498 , -1.813031  ],
       [-0.02795138,  1.4829755 ,  0.8779372 ],
       [-1.8868483 ,  0.2364595 ,  3.9750051 ],
       ...,
       [ 2.795917  , -0.13491254, -2.3476334 ],
       [-0.3330562 ,  2.5677974 , -0.39273626],
       [ 3.2823372 , -1.3956254 , -2.2047946 ]], dtype=float32)

In [40]:
y_pred = np.argmax(predictions.logits, axis=1)
y_pred

array([0, 1, 2, ..., 0, 1, 0])

In [41]:
accuracy_score(y_test, y_pred)

0.7496211783581425

In [42]:
def get_demo_result(demo_text, aspect):
    X_encoding = tokenizer(
        text=[demo_text],
        text_pair=[aspect],
        padding=True,
        truncation=True,
        max_length=42,
        return_token_type_ids=True
    )

    dataset = tf.data.Dataset.from_tensor_slices(
        dict(X_encoding)
    ).batch(BATCH_PARAM)

    prediction = model.predict(dataset)
    return tf.keras.activations.sigmoid(prediction.logits[0]).numpy(), np.argmax(prediction.logits, axis=1)

In [43]:
dict(zip(label_items, label_numbers))

{-1.0: 0, 0.0: 1, 1.0: 2}

In [49]:
get_demo_result("추천합니다. 좀 비싸긴 한데 너무 예뻐요. 털빠짐도 없구요.", "가격")

1/1 [==============================] - 0s 81ms/step


(array([0.8787629 , 0.57953066, 0.25647253], dtype=float32), array([0]))

In [50]:
get_demo_result("추천합니다. 좀 비싸긴 한데 너무 예뻐요. 털빠짐도 없구요.", "디자인")

1/1 [==============================] - 0s 75ms/step


(array([0.24960072, 0.5328826 , 0.9623261 ], dtype=float32), array([2]))

In [51]:
get_demo_result("추천합니다. 좀 비싸긴 한데 너무 예뻐요. 털빠짐도 없구요.", "품질") # 실패

1/1 [==============================] - 0s 104ms/step


(array([0.15856871, 0.91242886, 0.6451147 ], dtype=float32), array([1]))

In [52]:
get_demo_result("추천합니다. 좀 비싸긴 한데 너무 예뻐요. 털빠짐도 없구요.", "사이즈")

1/1 [==============================] - 0s 106ms/step


(array([0.09276251, 0.99156886, 0.11433197], dtype=float32), array([1]))

In [53]:
get_demo_result("추천합니다. 좀 비싸긴 한데 너무 예뻐요. 털빠짐도 없구요.", "기능")

1/1 [==============================] - 0s 72ms/step


(array([0.05377739, 0.993212  , 0.16296281], dtype=float32), array([1]))